# Instalações e imports

In [ ]:
%pip install shap
%pip install xgboost
%pip install mmh3
%pip install xgboost
# %pip install tensorflow

In [ ]:
import hashlib
import joblib
import matplotlib.pyplot as plt
import mmh3
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import tensorflow as tf
import xgboost as xgb
from IPython.core.display import HTML, display
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
  LabelEncoder,
  MinMaxScaler,
  OneHotEncoder,
  StandardScaler
)
from sklearn.tree import DecisionTreeRegressor
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, layers, regularizers, callbacks

<hr>

# Em Comum

## Descrição do trabalho
Este notebook realiza um experimento preliminar de contruir um modelo preditivo de regressão para valores de medicamentos a partir de uma base de dados que contém receitas médicas para medicamentos manipulados.
Os campos da base de dados original são:
- descricao - contém uma relação de principios ativos, dosagens e quantificações de dosagens, quantidade e descrição das formas farmacêuticas de medicamentos incluídos na formulação
- criado - tem a data/hora da criação da receita, possivelmente quando ela entrou no sistema da farmácia
- qtdInsumos - contém o número de insumos (chamados neste trabalho de princípio ativo) constantes na formulação prescrita e descrita na descrição
- calculado - contém o valor calculado (não sabemos se automaticamente no cadastro ou pela atendente que cadastrou)
- correto - contém o valor correto da medicação

Ao longo do notebook o experimento será descrito

In [ ]:
# carregamos o dataframe com as informações das receitas
df = pd.read_csv('./input/dados_preco.csv',sep=',')

### Tratamento Especial aos princípios ativos
Foi realizada uma análise exploratória preliminar no dataset candidato utilizando a ferramenta PowerBI. Esta análise se encontra em pdf no diretório deste projeto, bem como o arquivo pbix para ser aberto. Como não há certeza das versões de PowerBi pelo leitor, a análise também foi gerada em PDF.
Observou-se que o campo descrição continha alguns desafios comuns de mineração de textos e PNL. As ações para resolver esses desafios são tomadas a partir deste ponto do notebook.


In [ ]:
# criação de uma função para transformar a lista de principios ativos na descrição em um dicionario iterável em python,
# separando principio ativo e dosagens das quantidades e formas farmacêuticas
def parse_description(description):
  quantity, items = description.split("|")
  quantity = quantity.strip()
  items_list = [item.strip() for item in items.split(";")]
  items_dict = {str(i+1): item for i, item in enumerate(items_list)}
  return quantity, items_dict

# Exemplo de uso
description = "60 CAP | NAC  250MG; SILIMARINA  150MG; SAME  50MG"
quantidade, itens = parse_description(description)
print("Quantidade:", quantidade)
print("Itens:", itens)

In [ ]:
# adicionando ao Dataframe colunas separadas para principios ativos e quantidades
df[['quantidade', 'itens']] = df['descricao'].apply(lambda x: pd.Series(parse_description(x)))

In [ ]:
# criação de uma função para gerar um hash de identificação unitária para a receita, o que
# será util ao longo de todo o experimento
def generate_hash(description):
  return hashlib.md5(description.encode()).hexdigest()

In [ ]:
# Adicionando a coluna de hash ao dataframe
df['hash'] = df['descricao'].apply(generate_hash)

### Processamento de Linguagem Natural
O campo descrição da receita pode conter uma informação que consideramos muito valiosa, porque hipoteticamente, o fator determinante para o preço de um medicamento são os princípios ativos que estes contém.
Pelo motivo acima decidimos aprofundar ao máximo no tratamento do campo descrição.

In [ ]:
# copiando o primeiro DataFrame de forma preventiva (todas as antigas colunas, igual ao original)
df_original = df.copy()

# Criando o segundo DataFrame (apenas a coluna 'itens', expandida), mantendo o hash para não
# perder o link com a receita que originou os itens
item_rows = []
for _, row in df.iterrows():
  for key, value in row['itens'].items():
    item_rows.append({'hash': row['hash'], 'num_item': key, 'descricao_item': value})

df_itens = pd.DataFrame(item_rows)

In [ ]:
# separação do dataframe de itens do texto do principio ativo e da dosagem, de forma que o texto possa ser
# tratado especialmente
df_itens[['principio_ativo', 'dosagem']] = df_itens['descricao_item'].apply(lambda x: pd.Series([ ' '.join(x.split()[:-1]), x.split()[-1] ]))
df_itens['principio_ativo'] = df_itens['principio_ativo'].str.replace(',', '.')

In [ ]:
# visualizando o dataframe de itens
display(HTML(df_itens.head(10).to_html()))

#### Limpeza nos princípios ativos
Os princípios ativos serão então isolados em um dataframe separado, onde seus nomes serão deduplicados, e assim este dataframe será
submetido a um LLM para que se criem clusters de medicamentos. Conforme o leitor pode ver na Análise Exploratória em PowerBi, são 1632 princípios ativos descritos, que tem pequenas variações em linguagem natural que precisam ser corrigidas para um bom trabalho.

In [ ]:
# criação de um dataframe de princípios ativos
df_principios_ativos = df_itens[['principio_ativo']].copy()

# Agrupamento para contar quantos principios ativos iguais existem
df_principios_ativos['quantidade'] = df_principios_ativos.groupby('principio_ativo')['principio_ativo'].transform('size')

# Remover duplicatas
df_principios_ativos = df_principios_ativos.drop_duplicates(subset='principio_ativo').reset_index(drop=True)
# Orderar alfabeticamente
df_principios_ativos = df_principios_ativos.sort_values(by='principio_ativo').reset_index(drop=True)

# salvamento dos principios ativos para processamento do LLM
df_principios_ativos.to_excel('./output/principios_ativos.xlsx')

#### Utilização de LLM para clusterização dos princípios ativos
A clusterização de textos por similaridade não é uma funcionalidade nova, muito menos advinda dos LLMs, porém a título de demonstração optamos por utilizar a API da OpenAI com um prompt criado especificamente após a observaçao pela análise exploratória dos eventos mais comuns de diferença nas descrições armazenadas. O arquivo em principios_ativos.xlsx foi trabalho separadamente em um outro notebook também anexado neste trabalho. O nome deste notebook é llm_call_v2.

In [ ]:
# recuperação do cluster de medicamentos
cluster_medicamentos = pd.read_excel('./input/clusters.xlsx')

In [ ]:
# colunas retornadas no cluster medicamentos
cluster_medicamentos.columns

In [ ]:
# Adição do nome do cluster ao dataframe de itens
df_itens = df_itens.merge(cluster_medicamentos[['original', 'cluster']], left_on='principio_ativo', right_on='original', how='left')

In [ ]:
# Neste ponto estamos extraindo do item a unidade e a dosagem para ter a informação separada
df_itens['dose'] = df_itens['dosagem'].str.extract(r'(\d+[\.,]?\d*)')  # Extrai a parte numérica
df_itens['unidade'] = df_itens['dosagem'].str.extract(r'([a-zA-Z]+)')  # Extrai a parte textual (unidade de medida)

In [ ]:
# Salvamento do dataframe finalizado para análise exploratória no Power BI
df_itens.to_excel('./output/itens_finalizados.xlsx')

In [ ]:
# Avaliação das colunas do dataframe
df_itens.columns

In [ ]:
# simplificando o dataframe a titulo de facilitar colocar mais informação para linkar com o preprocessamento de PCA abaixo
df_itens_selecionado = df_itens[['hash', 'cluster', 'dose', 'unidade']]
pca_preprocessing_df = df_itens_selecionado.copy()

In [ ]:
df_itens_selecionado.columns

In [ ]:
df_matricial = df_itens_selecionado.copy()

# OneHotEncoder para o cluster (medicamentos)
ohe = OneHotEncoder(sparse_output=False)
clusters_encoded = ohe.fit_transform(df_matricial[['cluster']])
cluster_columns = ohe.get_feature_names_out(['cluster'])

# LabelEncoder para a unidade
le = LabelEncoder()
df_matricial.loc[:, 'unidade_encoded'] = le.fit_transform(df_matricial['unidade'])

# Criar DataFrame com matriz one-hot de princípios ativos
df_clusters = pd.DataFrame(clusters_encoded, columns=cluster_columns)
df_clusters['hash'] = df_matricial['hash']

# Agregar as matrizes de medicamentos, doses e unidades por hash
df_agg = df_clusters.groupby('hash').apply(lambda x: np.vstack(x[cluster_columns].to_numpy())).reset_index(name='matrix')
df_doses = df_matricial.groupby('hash')['dose'].apply(lambda x: np.array(x.tolist())).reset_index()
df_unidades = df_matricial.groupby('hash')['unidade_encoded'].apply(lambda x: np.array(x.tolist())).reset_index()

# Mesclar tudo no DataFrame final
final_df = df_agg.merge(df_doses, on='hash').merge(df_unidades, on='hash')

print(final_df.head())

In [ ]:
display(HTML(final_df.head(10).to_html()))

In [ ]:
final_df.columns

In [ ]:
print(final_df.dtypes)

### Transformando a quantidade de cápsulas em texto para números
Vamos agora em passos finais, isolar a quantidade de cápsulas em uma coluna para aproveitar este valor como um dos inputs no modelo.

In [ ]:
# Extrai apenas os números (inclui inteiros e decimais)
df_original['quantidade_prescrita'] = df_original['quantidade'].str.extract(r'(\d+\.?\d*)')

# Converte para número (float ou int) se necessário
df_original['quantidade_prescrita'] = pd.to_numeric(df_original['quantidade_prescrita'])

In [ ]:
df_original.columns

In [ ]:
df_treino = df_original.merge(final_df, on='hash')

### Ultimos tratamentos para construi o dataset para processamento

In [ ]:
# Transformando a data "criado" em unix timestamp numérico
df_treino['criado'] = pd.to_datetime(df_treino['criado'])

# Converte para timestamp em milissegundos
df_treino['criado'] = df_treino['criado'].astype('int64') // 10**6

In [ ]:
# retirando do dataframe campos que não são numéricos
df_final = df_treino.drop(['descricao', 'quantidade', 'itens'], axis=1)

In [ ]:
# Verificando as colunas finais
df_final.columns

In [ ]:
# verificando a integridade do dataframe final com o dataframe inicial
len(df_final) == len(df_original)

In [ ]:
#verificando se todos os tipos estão de acordo
print(df_final.dtypes)

In [ ]:
display(HTML(df_final.head(10).to_html()))

In [ ]:
df_final

In [ ]:
# 1 CARREGAMENTO DO DATASET
df_tensores = df_final.copy()

# 2️ TRATAMENTO DA COLUNA "CRIADO"
df_tensores["criado"] = pd.to_datetime(df_tensores["criado"], errors="coerce")  # Converter para datetime
df_tensores["ano"] = df_tensores["criado"].dt.year
df_tensores["mes"] = df_tensores["criado"].dt.month
df_tensores["dia"] = df_tensores["criado"].dt.day
df_tensores["hora"] = df_tensores["criado"].dt.hour

# 3 PRÉ-PROCESSAMENTO PADRÃO (Matrix, Padding, Normalização)
df_tensores["matrix"] = df_tensores["matrix"].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
df_tensores["dose"] = df_tensores["dose"].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_tensores["unidade_encoded"] = df_tensores["unidade_encoded"].apply(lambda x: eval(x) if isinstance(x, str) else x)

max_qtdInsumos = df_tensores["qtdInsumos"].max()

def pad_matrix(matrix, qtdInsumos, max_qtdInsumos):
  matrix = np.array(matrix)[:qtdInsumos]
  pad_size = max_qtdInsumos - matrix.shape[0]
  if pad_size > 0:
    padding = np.zeros((pad_size, 940))
    matrix = np.vstack([matrix, padding])
  return matrix

df_tensores["matrix"] = df_tensores.apply(lambda row: pad_matrix(row["matrix"], row["qtdInsumos"], max_qtdInsumos), axis=1)

df_tensores["dose"] = df_tensores.apply(lambda row: row["dose"][:row["qtdInsumos"]], axis=1)
df_tensores["unidade_encoded"] = df_tensores.apply(lambda row: row["unidade_encoded"][:row["qtdInsumos"]], axis=1)

df_tensores["dose"] = pad_sequences(df_tensores["dose"], maxlen=max_qtdInsumos, dtype="float32", padding="post").tolist()
df_tensores["unidade_encoded"] = pad_sequences(df_tensores["unidade_encoded"], maxlen=max_qtdInsumos, dtype="int32", padding="post").tolist()

# Normalização e salvamento do Scaler
scaler = MinMaxScaler()
df_tensores[["calculado", "quantidade_prescrita"]] = scaler.fit_transform(df_tensores[["calculado", "quantidade_prescrita"]])

joblib.dump(scaler, "./output/scaler.pkl") # Salvando o scaler

In [ ]:
df_determinante = df_tensores.copy()
df_determinante["matrix_real"] = df_determinante["matrix"].apply(lambda x: mmh3.hash(str(x.flatten().tolist()), signed=False) / (2**32))
df_determinante["dose_real"] = df_determinante["dose"].apply(lambda x: mmh3.hash(str(x), signed=False) / (2**32))
df_determinante["unidade_encoded_real"] = df_determinante["unidade_encoded"].apply(lambda x: mmh3.hash(str(x), signed=False) / (2**32))
df_determinante["criado"] = pd.to_datetime(df_determinante['criado']).astype('int64') // 10**6
df_determinante = df_determinante.drop(columns=['matrix', 'dose', 'unidade_encoded','ano','mes', 'dia', 'hora'])
df_determinante.columns

In [ ]:
df_tensores.head(10)

In [ ]:
df_determinante.head(10)

In [ ]:
pca_preprocessing_df.head(10)

<hr>

# PCA
notebook 2

## Enconding
Por se tratar de um caso de regressão, precisamos encontrar uma forma harmônica de representar os principios ativos e suas dosagens de maneira a conseguir transformar os mesmos, que são categóricos, em valores numéricos.
A execução da clusterização já foi feita com o objetivo de obviamente deduplicar, mas também simplificar e reduzir o número de categorias para realizar o encoding. Como pode ser visto na Análise Exploratória, reduzimos os princípios ativos de 1632 descrições para 940 clusters.<br>
Vários encoders são candidatos para serem usados neste momento. A título de tempo disponível utilizaremos uma abordagem onde a dose e a unidade serão embutidas no encoder.<br>
ATENÇÃO: o processo de encoder precisa ser pensado com muitíssimo cuidado. Neste experimento simplesmente implementamos a abordagem mais rápida possível devido ao tempo disponível para implementar e entregar. Muitas melhorias podem ser feitas neste processo, e seria de grande valor uma conversa 1:1 com o leitor para explicar melhor os prós, contras, riscos e mitigações deste enconder executado.
Enfim, neste caso o bom seria inimigo do ótimo, então seguimos em frente com foco na entrega de um MVP.

In [ ]:
# geracão de uma sigla com o nome do principio ativo tratando duplicados para simplificar o nome da coluna do encoder
siglas = {}
def gerar_sigla(cluster):
    partes = cluster.split()
    sigla = ''.join([p[:3].upper() for p in partes])  # Usa 3 primeiras letras de cada palavra
    original_sigla = sigla
    contador = 1
    while sigla in siglas:  # Se já existe, adiciona um número incremental
        sigla = original_sigla + str(contador)
        contador += 1
    siglas[sigla] = cluster
    return sigla
#  criação da sigla no dataframe

pca_preprocessing_df['cluster_sigla'] = pca_preprocessing_df['cluster'].apply(gerar_sigla)

# criação de um campo que será chave no encoder com a sigla+dose+unidade
pca_preprocessing_df['key'] = pca_preprocessing_df['cluster_sigla'] + '_' + pca_preprocessing_df['dose'].astype(str) + '_' + pca_preprocessing_df['unidade']
# Aplicar OneHotEncoder na chave reduzida
one_hot_encoder = OneHotEncoder(sparse_output=False)
encoded_values = one_hot_encoder.fit_transform(pca_preprocessing_df[['key']])

# Criar um DataFrame com as variáveis codificadas e nomes reduzidos
encoded_columns = one_hot_encoder.get_feature_names_out(['key'])
encoded_columns = [col.replace('key_', '') for col in encoded_columns]  # Remover prefixo desnecessário

encoded_df = pd.DataFrame(encoded_values, columns=encoded_columns)

# Concatenar os dados codificados ao DataFrame original
pca_preprocessing_encoded_df = pd.concat([pca_preprocessing_df[['hash']], encoded_df], axis=1)

# Agregar por 'hash' e usar 'max' para manter a presença dos valores
pca_preprocessing_aggregated_df = pca_preprocessing_encoded_df.groupby('hash').agg('max').reset_index()

# Exibir o resultado final
print(pca_preprocessing_aggregated_df)# Exibir o resultado final


In [ ]:
display(HTML(pca_preprocessing_aggregated_df.head(10).to_html()))

#### Redução de Dimensionalidade
Conforme pode ser visto acima, a combinatória de principio ativo (em cluster), dosagem e unidade gerou um encoder com 23355 colunas. Como prática neste campo da análise de regressão, precisamos executar uma redução de dimensionalidade para termos um dataset de treinamento, teste e validação viável para executar o experimento.
Tentamos então duas técnicas para escolher o melhor resultado:
- Análise de Componentes Principais (PCA)
- Truncated Singular Value Decomposition (TruncatedSVD)

<hr>

In [ ]:
X = pca_preprocessing_aggregated_df.drop(columns=['hash'])  # Remover a coluna 'hash' antes da redução

# Normalizar os dados antes do PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reduzindo para 50 componentes principais com Truncated SVD
svd = TruncatedSVD(n_components=50, random_state=42)
X_svd = svd.fit_transform(X_scaled)

# Testando PCA para comparação
pca = PCA(n_components=50, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Criar DataFrames com os componentes reduzidos
df_svd = pd.DataFrame(X_svd, columns=[f'comp_{i+1}' for i in range(50)])
df_pca = pd.DataFrame(X_pca, columns=[f'pca_{i+1}' for i in range(50)])

# Adicionar a coluna 'hash' de volta para identificação
df_svd.insert(0, 'hash', pca_preprocessing_aggregated_df['hash'])
df_pca.insert(0, 'hash', pca_preprocessing_aggregated_df['hash'])

In [ ]:
# Comparar a variância explicada
var_svd = np.sum(svd.explained_variance_ratio_)
var_pca = np.sum(pca.explained_variance_ratio_)

print(f"Variância explicada pelos 50 componentes:")
print(f"Truncated SVD: {var_svd:.2%}")
print(f"PCA: {var_pca:.2%}")

# Plotando os componentes principais dos dois métodos
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.scatterplot(x=df_svd['comp_1'], y=df_svd['comp_2'], alpha=0.5, ax=axes[0])
axes[0].set_title("Truncated SVD (Comp 1 vs Comp 2)")

sns.scatterplot(x=df_pca['pca_1'], y=df_pca['pca_2'], alpha=0.5, ax=axes[1])
axes[1].set_title("PCA (Comp 1 vs Comp 2)")

plt.show()

##### Vamos prosseguir o trabalho utilizando o PCA, simplesmente pela variancia semelhante e preferência pessoal pela técnica

### Transformando a quantidade de cápsulas em texto para números
Vamos agora em passos finais, isolar a quantidade de cápsulas em uma coluna para aproveitar este valor como um dos inputs no modelo.

In [ ]:
# Extrai apenas os números (inclui inteiros e decimais)
df_original['quantidade_prescrita'] = df_original['quantidade'].str.extract(r'(\d+\.?\d*)')

# Converte para número (float ou int) se necessário
df_original['quantidade_prescrita'] = pd.to_numeric(df_original['quantidade_prescrita'])

In [ ]:
df_original.columns

### Ultimos tratamentos para construi o dataset para processamento

In [ ]:
# Juntando o dataframe original com o dataframe representativo em PCA dos principios ativos
pca_preprocessed_df = pd.merge(df_original, df_pca, on='hash', how='inner')

In [ ]:
# Transformando a data "criado" em unix timestamp numérico
pca_preprocessed_df['criado'] = pd.to_datetime(pca_preprocessed_df['criado'])

# Converte para timestamp em milissegundos
pca_preprocessed_df['criado'] = pca_preprocessed_df['criado'].astype('int64') // 10**6

In [ ]:
# retirando do dataframe campos que não são numéricos
pca_preprocessed_df = pca_preprocessed_df.drop(['descricao', 'quantidade', 'itens'], axis=1)

In [ ]:
# Verificando as colunas finais
pca_preprocessed_df.columns

In [ ]:
# Verificando a integridade do dataframe final com o dataframe inicial
len(pca_preprocessed_df) == len(df)

In [ ]:
# verificando se todos os tipos estão de acordo
print(pca_preprocessed_df.dtypes)

In [ ]:
pca_preprocessed_df.head(10)

### Treinamento
Posto que atingimos um dataset com número de parâmetros de acordo para o treinamento de um modelo de regressão, vamos seguir o experimento proponto três modelos distintos para escolher, em teste, qual deles tem a melhor performance.

#### Train Test Split
Com retirada antecipada de 2000 elementos aleatórios para validação posterior

In [ ]:
# Passo 1: Separar as 2000 linhas para validação conforme orientado 
pca_receitas_validation_df = pca_preprocessed_df.sample(n=2000, random_state=42)

# O restante do dataset será usado para treinamento e teste
pca_receitas_train_test_df = pca_preprocessed_df.drop(pca_receitas_validation_df.index)

# Passo 2: Divisão entre treino e teste (80% treino, 20% teste)
X = pca_receitas_train_test_df.drop(columns=['correto', 'hash'])
y = pca_receitas_train_test_df['correto']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Normalização

In [ ]:
# Passo 3: Pré-processamento - Normalização dos dados numéricos
scaler = StandardScaler()

# Normalizando os dados de treino e teste
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### Árvore de Descisão

In [ ]:
# Modelo 1: Árvore de Decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - Árvore de Decisão
y_pred_tree = tree_model.predict(X_test_scaled)
rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))
mae_tree = mean_absolute_error(y_test, y_pred_tree)
print(f"Árvore de Decisão - RMSE: {rmse_tree:.4f}, MAE: {mae_tree:.4f}")

##### Random Forest

In [ ]:
# Modelo 2: Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - Random Forest
y_pred_rf = rf_model.predict(X_test_scaled)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest - RMSE: {rmse_rf:.4f}, MAE: {mae_rf:.4f}")

##### XGBoost

In [ ]:
# Modelo 3: XGBoost Regressor
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - XGBoost
y_pred_xgb = xgb_model.predict(X_test_scaled)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost - RMSE: {rmse_xgb:.4f}, MAE: {mae_xgb:.4f}")

In [ ]:
# Passo 8: Explicabilidade com SHAP (XGBoost)
explainer = shap.Explainer(xgb_model, X_train_scaled)
shap_values = explainer(X_test_scaled)

# Visualizando o gráfico de importância das variáveis
shap.summary_plot(shap_values, X_test)

# Passo 9: Usando o modelo XGBoost para prever no conjunto de validação
X_validation = pca_receitas_validation_df.drop(columns=['correto', 'hash'])
X_validation_scaled = scaler.transform(X_validation)

y_validation_pred = xgb_model.predict(X_validation_scaled)

# Visualizando as predições para o conjunto de validação
pca_receitas_validation_df['predito'] = y_validation_pred

# Se desejar salvar as predições para análises futuras
pca_receitas_validation_df[['hash', 'correto', 'predito']].to_csv('./output/predicoes_validacao_pca.csv', index=False)

# Tensores
notebook 4 (rede neural)

In [ ]:
# SEPARAÇÃO DOS DADOS
df_tensores_validation = df_tensores.sample(n=2000, random_state=42)
df_tensores_train_test = df_tensores.drop(df_tensores_validation.index)
validation_hashes = df_tensores_validation["hash"].values

X_matrix = np.stack(df_tensores_train_test["matrix"].values)
X_dose = np.stack(df_tensores_train_test["dose"].values)
X_unidade = np.stack(df_tensores_train_test["unidade_encoded"].values)

X_numeric = df_tensores_train_test[["qtdInsumos", "calculado", "quantidade_prescrita", "ano", "mes", "dia", "hora"]].values

y = df_tensores_train_test["correto"].values

X = np.hstack([
  X_numeric,
  X_matrix.reshape(len(X_matrix), -1),
  X_dose,
  X_unidade
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df_tensores_train_test.head(10)

In [ ]:
## 1º - MAE: 79.2750 RMSE: 272.077

# # 5️⃣ CONSTR1els.Sequential([
#     layers.Dense(256, activation="relu", input_shape=(X_train.shape[1],)),
#     layers.Dropout(0.3),
#     layers.Dense(128, activation="relu"),
#     layers.Dropout(0.2),
#     layers.Dense(64, activation="relu"),
#     layers.Dense(1)
# ])

# model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# # 6️⃣ TREINAMENTO
# model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# # 7️⃣ SALVAR O MODELO
# model.save("./output/models/modelo_tensorflow.keras")  # Salvar modelo no formato TensorFlow

In [ ]:
## 2º - MAE: 81.1083 RMSE: 274.1747

# from tensorflow.keras import models, layers, regularizers, callbacks

# model = models.Sequential([
#   layers.Dense(512, activation="relu", input_shape=(X_train.shape[1],),
#     kernel_regularizer=regularizers.l2(0.001)),
#   layers.BatchNormalization(),
#   layers.Dropout(0.3),
  
#   layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
#   layers.BatchNormalization(),
#   layers.Dropout(0.3),
  
#   layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
#   layers.BatchNormalization(),
#   layers.Dropout(0.2),
  
#   layers.Dense(64, activation="relu"),
#   layers.BatchNormalization(),
#   layers.Dropout(0.1),
  
#   layers.Dense(1)
# ])

# model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# # Definindo callbacks para melhorar o treinamento
# early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
# reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

# model.fit(
#   X_train, y_train,
#   epochs=100,  # aumentando o número de épocas para dar mais tempo de aprendizado
#   batch_size=64,
#   validation_data=(X_test, y_test),
#   callbacks=[early_stop, reduce_lr]
# )

# model.save("./output/models/modelo_tensorflow_refinado.keras")

In [ ]:
# 3º - MAE: 68.8011  RMSE: 255.5577 Rede neural multilayer perceptron


model = models.Sequential([
  layers.Dense(1024, activation="relu", input_shape=(X_train.shape[1],),
    kernel_regularizer=regularizers.l2(0.001)),
  layers.BatchNormalization(),
  layers.Dropout(0.3),

  layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
  layers.BatchNormalization(),
  layers.Dropout(0.3),

  layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
  layers.BatchNormalization(),
  layers.Dropout(0.3),

  layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
  layers.BatchNormalization(),
  layers.Dropout(0.2),

  layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
  layers.BatchNormalization(),
  layers.Dropout(0.2),

  layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
  layers.BatchNormalization(),
  layers.Dropout(0.1),

  layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

model.fit(
  X_train, y_train,
  epochs=200,
  batch_size=64,
  validation_data=(X_test, y_test),
  callbacks=[early_stop, reduce_lr]
)

model.save("./output/models/modelo_tensorflow_profundo.keras")

In [ ]:
## 4º - MAE: 82.1332  RMSE: 263.7608

# from tensorflow.keras import models, layers, regularizers, callbacks
# from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, add, LeakyReLU

# # Entrada
# input_layer = Input(shape=(X_train.shape[1],))

# # Primeira camada densa
# x = Dense(1024, kernel_regularizer=regularizers.l2(0.001))(input_layer)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.1)(x)
# x = Dropout(0.3)(x)

# # Bloco Residual 1
# res = Dense(512, kernel_regularizer=regularizers.l2(0.001))(x)
# res = BatchNormalization()(res)
# res = LeakyReLU(alpha=0.1)(res)
# res = Dropout(0.3)(res)

# # Atalho para ajustar a dimensão
# shortcut = Dense(512, kernel_regularizer=regularizers.l2(0.001))(x)
# shortcut = BatchNormalization()(shortcut)

# # Soma do bloco residual com o atalho
# x = add([res, shortcut])
# x = LeakyReLU(alpha=0.1)(x)

# # Bloco Residual 2
# res = Dense(256, kernel_regularizer=regularizers.l2(0.001))(x)
# res = BatchNormalization()(res)
# res = LeakyReLU(alpha=0.1)(res)
# res = Dropout(0.3)(res)

# # Atalho para o segundo bloco
# shortcut = Dense(256, kernel_regularizer=regularizers.l2(0.001))(x)
# shortcut = BatchNormalization()(shortcut)

# x = add([res, shortcut])
# x = LeakyReLU(alpha=0.1)(x)

# # Camadas subsequentes sem conexão residual
# x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)

# x = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)

# x = Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
# x = BatchNormalization()(x)
# x = Dropout(0.1)(x)

# # Camada de saída com ativação linear para regressão
# output = Dense(1, activation="linear")(x)

# model = models.Model(inputs=input_layer, outputs=output)

# model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# # Callbacks para monitoramento
# early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
# reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

# model.fit(
#   X_train, y_train,
#   epochs=200,
#   batch_size=64,
#   validation_data=(X_test, y_test),
#   callbacks=[early_stop, reduce_lr]
# )

# model.save("./output/models/modelo_tensorflow_residual.keras")

In [ ]:
# # 5º - MAE: 77.7278  RMSE: 274.1434

# from tensorflow.keras import models, layers, regularizers, callbacks
# from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, add, LeakyReLU

# # Entrada
# input_layer = Input(shape=(X_train.shape[1],))

# x = Dense(1024, kernel_regularizer=regularizers.l2(0.0005))(input_layer)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.01)(x)
# x = Dropout(0.2)(x)

# # Primeiro bloco residual
# res = Dense(512, kernel_regularizer=regularizers.l2(0.0005))(x)
# res = BatchNormalization()(res)
# res = LeakyReLU(alpha=0.01)(res)
# res = Dropout(0.2)(res)

# shortcut = Dense(512, kernel_regularizer=regularizers.l2(0.0005))(x)
# shortcut = BatchNormalization()(shortcut)

# x = add([res, shortcut])
# x = LeakyReLU(alpha=0.01)(x)

# # Segundo bloco residual
# res = Dense(256, kernel_regularizer=regularizers.l2(0.0005))(x)
# res = BatchNormalization()(res)
# res = LeakyReLU(alpha=0.01)(res)
# res = Dropout(0.2)(res)

# shortcut = Dense(256, kernel_regularizer=regularizers.l2(0.0005))(x)
# shortcut = BatchNormalization()(shortcut)

# x = add([res, shortcut])
# x = LeakyReLU(alpha=0.01)(x)

# # Camadas adicionais sem conexão residual
# x = Dense(128, kernel_regularizer=regularizers.l2(0.0005))(x)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.01)(x)
# x = Dropout(0.15)(x)

# x = Dense(64, kernel_regularizer=regularizers.l2(0.0005))(x)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.01)(x)
# x = Dropout(0.15)(x)

# x = Dense(32, kernel_regularizer=regularizers.l2(0.0005))(x)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.01)(x)
# x = Dropout(0.1)(x)

# # Camada de saída (regressão)
# output = Dense(1, activation="linear")(x)

# model = models.Model(inputs=input_layer, outputs=output)

# model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
# reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-7)

# model.fit(
#   X_train, y_train,
#   epochs=300,
#   batch_size=64,
#   validation_data=(X_test, y_test),
#   callbacks=[early_stop, reduce_lr]
# )

# model.save("./output/models/modelo_tensorflow_residual_ajustado.keras")

In [ ]:

# 8️ AVALIAÇÃO
def evaluate_model(model, X, y, dataset_name):
  y_pred = model.predict(X)
  mae = mean_absolute_error(y, y_pred)
  rmse = np.sqrt(mean_squared_error(y, y_pred))
  print(f"\n Avaliação no conjunto {dataset_name}:")
  print(f"    MAE:  {mae:.4f}")
  print(f"    RMSE: {rmse:.4f}")
  return mae, rmse

# Avaliação no conjunto de teste
evaluate_model(model, X_test, y_test, "TESTE")

# Avaliação no conjunto de validação
X_validation_matrix = np.stack(df_tensores_validation["matrix"].values)
X_validation_dose = np.stack(df_tensores_validation["dose"].values)
X_validation_unidade = np.stack(df_tensores_validation["unidade_encoded"].values)

X_validation_numeric = df_tensores_validation[["qtdInsumos", "calculado", "quantidade_prescrita", "ano", "mes", "dia", "hora"]].values

X_validation = np.hstack([
  X_validation_numeric,  
  X_validation_matrix.reshape(len(X_validation_matrix), -1),  
  X_validation_dose,  
  X_validation_unidade
])

evaluate_model(model, X_validation, df_tensores_validation["correto"].values, "VALIDAÇÃO")

In [ ]:
# Geração dos valores preditos para o conjunto de validação
y_validation_pred = model.predict(X_validation).flatten()

# Criação do DataFrame com valores reais e preditos
df_val_resultado = df_tensores_validation.copy()
df_val_resultado["predito"] = y_validation_pred

# Se quiser salvar em CSV
df_val_resultado[["hash", "criado", "qtdInsumos", "calculado", "correto", "quantidade_prescrita", "predito"]].to_csv("./output/precicoes_validacao_tensores.csv", index=False, sep="|")

#### Resultado da otimização
Não cremos que a otimização possa trazer resultados muito diferentes, então finalizamos o experimento.
O arquivo Power BI de análise exploratória descreve o comportamento dos dados de validação.

<hr>

# Determinantes
notebook 5

#### Train Test Split
Com retirada antecipada de 2000 elementos aleatórios para validação posterior

In [ ]:
# Passo 1: Separar as 2000 linhas para validação conforme orientado
determinante_preprocessed_df = df_determinante.copy()
df_determinante_receitas_validation = determinante_preprocessed_df.sample(n=2000, random_state=42)

# O restante do dataset será usado para treinamento e teste
df_determinante_receitas_train_test = determinante_preprocessed_df.drop(df_determinante_receitas_validation.index)

# Passo 2: Divisão entre treino e teste (80% treino, 20% teste)
X = df_determinante_receitas_train_test.drop(columns=['correto', 'hash'])
y = df_determinante_receitas_train_test['correto']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.head(10)

#### Normalização

In [ ]:
# Passo 3: Pré-processamento - Normalização dos dados numéricos
scaler = StandardScaler()

# Normalizando os dados de treino e teste
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### Árvore de Descisão

In [ ]:
# Modelo 1: Árvore de Decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - Árvore de Decisão
y_pred_tree = tree_model.predict(X_test_scaled)
rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))
mae_tree = mean_absolute_error(y_test, y_pred_tree)
print(f"Árvore de Decisão - RMSE: {rmse_tree:.4f}, MAE: {mae_tree:.4f}")

##### Random Forest

In [ ]:
# Modelo 2: Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - Random Forest
y_pred_rf = rf_model.predict(X_test_scaled)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest - RMSE: {rmse_rf:.4f}, MAE: {mae_rf:.4f}")

##### XGBoost

In [ ]:
# Modelo 3: XGBoost Regressor
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - XGBoost
y_pred_xgb = xgb_model.predict(X_test_scaled)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost - RMSE: {rmse_xgb:.4f}, MAE: {mae_xgb:.4f}")

In [ ]:
# Passo 8: Explicabilidade com SHAP (XGBoost)
explainer = shap.Explainer(xgb_model, X_train_scaled)
shap_values = explainer(X_test_scaled)

# Visualizando o gráfico de importância das variáveis
shap.summary_plot(shap_values, X_test)

# Passo 9: Usando o modelo XGBoost para prever no conjunto de validação
X_validation = df_determinante_receitas_validation.drop(columns=['correto', 'hash'])
X_validation_scaled = scaler.transform(X_validation)

y_validation_pred = xgb_model.predict(X_validation_scaled)

# Visualizando as predições para o conjunto de validação
df_determinante_receitas_validation['predito'] = y_validation_pred

# Se desejar salvar as predições para análises futuras
df_determinante_receitas_validation[['hash', 'correto', 'predito']].to_csv('./output/predicoes_validacao_determinante.csv', index=False)

<hr>

# Simplificado (sem calculado)
notebook 6

#### Train Test Split
Com retirada antecipada de 2000 elementos aleatórios para validação posterior

In [ ]:
# Passo 1: Removendo coluna calculado
simplificado_preprocessing_df = df_determinante.drop(columns=['calculado'])

# Passo 1: Separar as 2000 linhas para validação conforme orientado
df_simplificado_receitas_validation = simplificado_preprocessing_df.sample(n=2000, random_state=42)

# O restante do dataset será usado para treinamento e teste
df_simplificado_receitas_train_test = simplificado_preprocessing_df.drop(df_simplificado_receitas_validation.index)

# Passo 2: Divisão entre treino e teste (80% treino, 20% teste)
X = df_simplificado_receitas_train_test.drop(columns=['correto', 'hash'])
y = df_simplificado_receitas_train_test['correto']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.head(10)

#### Normalização

In [ ]:
# Passo 3: Pré-processamento - Normalização dos dados numéricos
scaler = StandardScaler()

# Normalizando os dados de treino e teste
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### Árvore de Descisão

In [ ]:
# Modelo 1: Árvore de Decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - Árvore de Decisão
y_pred_tree = tree_model.predict(X_test_scaled)
rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))
mae_tree = mean_absolute_error(y_test, y_pred_tree)
print(f"Árvore de Decisão - RMSE: {rmse_tree:.4f}, MAE: {mae_tree:.4f}")

##### Random Forest

In [ ]:
# Modelo 2: Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - Random Forest
y_pred_rf = rf_model.predict(X_test_scaled)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest - RMSE: {rmse_rf:.4f}, MAE: {mae_rf:.4f}")

##### XGBoost

In [ ]:
# Modelo 3: XGBoost Regressor
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Avaliação do modelo - XGBoost
y_pred_xgb = xgb_model.predict(X_test_scaled)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost - RMSE: {rmse_xgb:.4f}, MAE: {mae_xgb:.4f}")

In [ ]:
# Passo 8: Explicabilidade com SHAP (XGBoost)
explainer = shap.Explainer(xgb_model, X_train_scaled)
shap_values = explainer(X_test_scaled)

# Visualizando o gráfico de importância das variáveis
shap.summary_plot(shap_values, X_test)

# Passo 9: Usando o modelo XGBoost para prever no conjunto de validação
X_validation = df_simplificado_receitas_validation.drop(columns=['correto', 'hash'])
X_validation_scaled = scaler.transform(X_validation)

y_validation_pred = xgb_model.predict(X_validation_scaled)

# Visualizando as predições para o conjunto de validação
df_simplificado_receitas_validation['predito'] = y_validation_pred

# Se desejar salvar as predições para análises futuras
df_simplificado_receitas_validation[['hash', 'correto', 'predito']].to_csv('./output/predicoes_validacao_simplificado.csv', index=False)

<hr>

## TODO
- Nos treinamentos fora dos comuns, alterar os nomes e utilizar os dataframes que foram printados ao fim do bloco "Em Comum"
- Treinar o PCA com os 3 algoritmos, e criar o gráfico SHAP
- Rede neural e processar
- Determinante, processar e criar o gráfico SHAP